# debug web
#https://pythononpow.medium.com/debugging-a-tornado-pythononwheels-web-application-with-vscode-d9f5c5e5603f

In [1]:
# {
#     // 使用 IntelliSense 了解相关属性。 
#     // 悬停以查看现有属性的描述。
#     // 欲了解更多信息，请访问: https://go.microsoft.com/fwlink/?linkid=830387
#     "version": "0.2.0",
#     "configurations": [
        
#         {  
#             "name": "Python: PoW",  
#             "type": "python",  
#             "request": "launch",  
#             "program": "${workspaceFolder}/instock/web/web_service.py",  
#             "args": [],  
#             "console": "integratedTerminal"  
#         }
            
#     ]
# }

In [2]:
import logging
import os
import pymysql
from sqlalchemy import create_engine
from sqlalchemy.types import NVARCHAR
from sqlalchemy import inspect
import datetime

import sys

# stdout=sys.stdout
sys.path.append('../../')
from JohnsonUtil import commonTips as cct
from JSONData import tdx_data_Day as tdd

# 初始化db

In [3]:
db_host = "192.168.50.60"  # 数据库服务主机
db_user = "root"  # 数据库访问用户
db_password = "mariadb"  # 数据库访问密码
db_database = "instockdb"  # 数据库名称
db_port = 3306  # 数据库服务端口
db_charset = "utf8mb4"  # 数据库字符集

# 使用环境变量获得数据库,docker -e 传递
_db_host = os.environ.get('db_host')
if _db_host is not None:
    db_host = _db_host
_db_user = os.environ.get('db_user')
if _db_user is not None:
    db_user = _db_user
_db_password = os.environ.get('db_password')
if _db_password is not None:
    db_password = _db_password
_db_database = os.environ.get('db_database')
if _db_database is not None:
    db_database = _db_database
_db_port = os.environ.get('db_port')
if _db_port is not None:
    db_port = int(_db_port)

MYSQL_CONN_URL = "mysql+pymysql://%s:%s@%s:%s/%s?charset=%s" % (
    db_user, db_password, db_host, db_port, db_database, db_charset)

MYSQL_CONN_DBAPI = {'host': db_host, 'user': db_user, 'password': db_password, 'database': db_database,
                    'charset': db_charset, 'port': db_port, 'autocommit': True}



## def func

In [4]:
def write_code_to_blk(codew, blk='060'):
    block_path = tdd.get_tdx_dir_blocknew() + '%s.blk' % (blk)
    write_blk = 'n'
    write_blk = cct.cct_raw_input("write blk [Y] or [N]:")
    if write_blk == 'y' or write_blk == 'Y':
        hdf5_wri = cct.cct_raw_input(
            "Rewrite code [Y] or append [N](defalut:N):")
        if hdf5_wri == 'y' or hdf5_wri == 'Y':
            append_status = False
        else:
            append_status = True
        if len(codew) > 3:
            cct.write_to_blocknew(block_path, codew, append_status,
                                  doubleFile=False, keep_last=0)
            print("write:%s block_path:%s" % (len(codew), block_path))
        else:
            print("write No:%s block_path:%s" % (len(codew), block_path))
            # ("write error:%s block_path:%s" % (len(codew), block_path))
    return True

#allday
def groupby_count_code(df):
    df['couts'] = df.groupby('code')['code'].transform('count')
    if 'date' in df.columns:
        df = df.sort_values(by=['couts','date'],ascending=[0,1])
    else:
        df = df.sort_values(by=['couts'],ascending=[0])
    return df

In [5]:
def get_today(sep='-'):
    TODAY = datetime.date.today()
    fstr = "%Y" + sep + "%m" + sep + "%d"
    today = TODAY.strftime(fstr)
    return today

#前一日
def day_last_days(daynow,last=-1):
    return str(datetime.datetime.strptime(daynow, '%Y-%m-%d').date() + datetime.timedelta(last))

#上一个工作日
def last_tddate(days=1):
    # today = datetime.datetime.today().date() + datetime.timedelta(-days)
    if days is None:
        return days
    today = datetime.datetime.today().date()
    # log.debug("today:%s " % (today))
    # return str(today)

    def get_work_day(today):
        day_n = int(today.strftime("%w"))
        if day_n == 0:
            lastd = today + datetime.timedelta(-2)
            # log.debug("0:%s" % lastd)
        elif day_n == 1:
            lastd = today + datetime.timedelta(-3)
            # log.debug("1:%s" % lastd)
        else:
            lastd = today + datetime.timedelta(-1)
            # log.debug("2-6:%s" % lastd)
        return lastd
        # if days==0:
        # return str(lasd)
    lastday = today
    for x in range(int(days)):
        # print x
        lastday = get_work_day(today)
        today = lastday
    return str(lastday)

# 通过数据库链接 engine
def engine():
    return create_engine(MYSQL_CONN_URL)


def engine_to_db(to_db):
    _engine = create_engine(MYSQL_CONN_URL.replace(f'/{db_database}?', f'/{to_db}?'))
    return _engine


# DB Api -数据库连接对象connection，有游标
def conn_with_cursor():
    return conn_not_cursor().cursor()

# DB Api -数据库连接对象connection，无游标
def conn_not_cursor():
    try:
        _db = pymysql.connect(**MYSQL_CONN_DBAPI)
    except Exception as e:
        logging.error(f"database.conn_not_cursor处理异常：{MYSQL_CONN_DBAPI}{e}")
    return _db

# 查询数据
def executeSqlFetch(sql, params=()):
    with conn_with_cursor() as db:
        try:
            db.execute(sql, params)
        except Exception as e:
            logging.error(f"database.executeSqlFetch处理异常：{sql}{e}")

        result = db.fetchall()
        db.close()
        return result
    
# 计算数量
def executeSqlCount(sql, params=()):
    with conn_with_cursor() as db:
        try:
            db.execute(sql, params)
        except Exception as e:
            logging.error(f"database.select_count计算数量处理异常：{e}")

        result = db.fetchall()
        db.close()
        # 只有一个数组中的第一个数据
        if len(result) == 1:
            return int(result[0][0])
        else:
            return 0

In [6]:
try:
    with pymysql.connect(**MYSQL_CONN_DBAPI) as mydb:
        mydb.cursor().execute(" select 1 ")
except Exception as e:
        logging.error("执行信息：数据库不存在，将创建。")

## 放量上涨table 
cn_stock_strategy_entercn_stock_strategy_enter

In [7]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')


def panda_df(conn,sql):
    df = pd.read_sql(sql, conn)
    print(df.shape)
    return df

from pandas import DataFrame as df
def exe_sql_select(conn,sql):
	cur = conn.cursor() 
	data = None
	try: # 使用异常处理，以防程序无法正常运行
		cur.execute(sql) 
		data = df(cur.fetchall(), columns = [col[0] for col in cur.description]) 
	except Exception as e:
		conn.rollback() # 发生错误时回滚
		print('事务处理失败', e)
	else:
		# conn.commit() # 事务提交
		print('事务处理成功', cur.rowcount)
	# cur.close()
	return data



### 读取SQL数据

In [13]:
conn = pymysql.connect(**MYSQL_CONN_DBAPI)

_selcolall = f''' `date`,`code`,`name`,`rate_1` '''
_selcol = f''' `code`,`name`,`rate_1` '''

_table_name = 'cn_stock_strategy_enter'

lastday = last_tddate(1)

today_now = last_tddate(0)



sql = f'''SELECT '{_selcol}' FROM `{_table_name}` WHERE `date` = '{today_now}' and 
								`kdjk` >= 80 and `kdjd` >= 70 and `kdjj` >= 100 and `rsi_6` >= 80 and 
								`cci` >= 100 and `cr` >= 300 and `wr_6` >= -20 and `vr` >= 160'''

sql_now_price = f'''SELECT {_selcol} FROM `{_table_name}` WHERE `date` = '{lastday}' AND rate_1 > 0 '''

sql_now_today = f'''SELECT {_selcol} FROM `{_table_name}` WHERE `date` = '{today_now}' '''

# sql_now_today = f'''SELECT {_selcol} FROM `{_table_name}` WHERE `date` = '{lastday}' '''


sql_now_last7day = f'''SELECT {_selcolall} FROM `{_table_name}` WHERE `date` >= '{last_tddate(7)}' '''

sql_now_alltoday = f'''SELECT {_selcolall} FROM `{_table_name}`  '''

sql_tables_name =f'''select table_name from information_schema.tables where table_schema=`{db_database}` '''
# sql_columns = f'''select COLUMN_NAME from information_schema.COLUMNS where TABLE_SCHEMA=`{db_database}` and table_name = `{_table_name}`'''

sql_tables_name ='SHOW DATABASES'
# print('当前库下所有表的名称')
# sql_columns = "select COLUMN_NAME from information_schema.COLUMNS where table_name = '%s'"%(_table_name)
sql_columns = f'''select COLUMN_NAME from information_schema.COLUMNS where table_name = '{_table_name}' '''
print(sql_now_price)
print(sql_now_today)

mycursor = conn.cursor()
# mycursor.execute(sql_now_price)
# print(_selcol)
# count = 0
# for field in mycursor.fetchall():
# 		# print(field[0],end='  ')
# 		# print(field[1:4])
# 		# print(field)
# 		count +=1
# print("all:%s"%(count))


#lastday 
# df = exe_sql_select(conn,sql_now_price)

#today:
df = exe_sql_select(conn,sql_now_today)

# mycursor2 = pymysql.connect(**MYSQL_CONN_DBAPI)
df2 = panda_df(conn,sql_now_today)

# df3all = panda_df(conn,sql_now_alltoday)

dflast7d = panda_df(conn,sql_now_last7day)

# mycursor.execute(sql_columns)
# mycursor.execute(sql_now)

# myresult = mycursor.fetchall()
# for x in myresult:
#   print(x)
	
conn.close()

SELECT  `code`,`name`,`rate_1`  FROM `cn_stock_strategy_enter` WHERE `date` = '2023-09-26' AND rate_1 > 0 
SELECT  `code`,`name`,`rate_1`  FROM `cn_stock_strategy_enter` WHERE `date` = '2023-09-27' 
事务处理失败 'DataFrame' object is not callable
(27, 3)
(167, 4)


In [9]:
#today
print(df2[:2],df2.shape)
vol_code = df2.code.tolist()[:2]
#allday
df2[:2]
dflast7d[:2]


     code  name  rate_1
0  000813  德展健康    9.93
1  002300  太阳电缆    3.94 (27, 3)


,date,code,name,rate_1
0,2023-09-18,000610,西安旅游,-4.38
1,2023-09-18,002184,海得控制,-1.12


In [10]:
# df3all = groupby_count_code(df3all)
dflast7d = groupby_count_code(dflast7d)

### show couts

In [11]:
filter_date = '2023-04-28'
filter_date = today_now
# last_tddate(0)
# filter_date = lastday

In [12]:
df7multiIndex = dflast7d.reset_index().set_index(['code','date'])
df7multiIndex[:5]
df7tail = df7multiIndex.groupby(level=[0]).tail(1)
# df7tail.reset_index().code
df7tail =  df7tail.sort_values(by=['rate_1'],ascending=[0])

### show Today

In [14]:
df7tail =  df7tail.reset_index().set_index('code')
# df7tail[ df7tail.rate_1 ] 
df7tail[(df7tail.couts >1) & (df7tail.date >= datetime.datetime.strptime(filter_date, '%Y-%m-%d').date())][:6]

,date,index,name,rate_1,couts
code,,,,,
300735,2023-09-21,126,光弘科技,19.97,2
300200,2023-09-21,124,高盟新材,16.19,2
600355,2023-09-21,128,精伦电子,10.08,2


In [15]:
df7tail[(df7tail.date >= datetime.datetime.strptime(filter_date, '%Y-%m-%d').date()) ][:10]

,date,index,name,rate_1,couts
code,,,,,
300735,2023-09-21,126,光弘科技,19.97,2
300200,2023-09-21,124,高盟新材,16.19,2
300504,2023-09-21,125,天邑股份,10.23,1
600355,2023-09-21,128,精伦电子,10.08,2
002333,2023-09-21,121,罗普斯金,10.05,1
003015,2023-09-21,123,日久光电,10.01,1
000158,2023-09-21,120,常山北明,9.99,1
300968,2023-09-21,127,格林精密,7.87,1
601872,2023-09-21,129,招商轮船,7.29,1


In [16]:
# df7tail =  df7tail.sort_values(by=['rate_1'],ascending=[0])
# df7tail

### 写入放量上涨061

In [17]:
# df7code = df7tail[df7tail.couts >1].index.get_level_values('code')
# filter_date = '2023-04-28' couts >1

df7code = df7tail[(df7tail.couts >1) & (df7tail.date >= datetime.datetime.strptime(filter_date, '%Y-%m-%d').date())].index

#nowday
df7code = df7tail[(df7tail.date >= datetime.datetime.strptime(filter_date, '%Y-%m-%d').date()) ].index
df7code
len(df7code)

10

In [18]:

# hdf5_wri = cct.cct_raw_input("rewrite code [Y] or append [N]:")
# if len(hdf5_wri) == 0:
#     pass
# elif hdf5_wri == 'y' or hdf5_wri == 'Y':
#     append_status = False
# else:
#     append_status = True
# df2.code.values
#today:
# write_code_to_blk(df7code,'061')

# write_code_to_blk(df7code,'062')

# write_code_to_blk(df7code,'064')


## 读取cn_stock_strategy_keep_increasing

In [19]:
# conn = pymysql.connect(**MYSQL_CONN_DBAPI)

# _selcolall = f''' `date`,`code`,`name`,`rate_1` '''
# _selcol = f''' `code`,`name`,`rate_1` '''
_selcol = f''' *'''
_table_name = 'cn_stock_strategy_keep_increasing'
# date = last_tddate(1)
today_now = last_tddate(0)
print(today_now)
# conn = pymysql.connect(**MYSQL_CONN_DBAPI)
start_d = '2023-05-08'
start_d = last_tddate(5)
# sql_now_today = f'''SELECT {_selcol} FROM `{_table_name}` WHERE `date` >= '{start_d}' '''

sql_now_today = f'''SELECT {_selcol} FROM `{_table_name}` WHERE `date` = '{today_now}' '''

#mutildate
sql_now_allday = f'''SELECT {_selcol} FROM `{_table_name}` WHERE `date` > '{today_now}'- INTERVAL 5 DAY '''

# mycursor2 = pymysql.connect(**MYSQL_CONN_DBAPI)
df_increas = panda_df(engine(),sql_now_today)

df_increasA = panda_df(engine(),sql_now_allday)

2023-09-21
(4, 103)
(15, 103)


In [20]:
df_increas = df_increas.sort_values(by=['rate_1'],ascending=[1])
df_increas

,date,code,name,rate_1,rate_2,rate_3,rate_4,rate_5,rate_6,rate_7,...,rate_91,rate_92,rate_93,rate_94,rate_95,rate_96,rate_97,rate_98,rate_99,rate_100
2,2023-09-21,600243,青海华鼎,4.54,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,2023-09-21,601872,招商轮船,7.29,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
0,2023-09-21,002855,捷荣技术,10.00,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,2023-09-21,300504,天邑股份,10.23,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


## 写入周063

In [21]:
# write_code_to_blk(df_increas.code.tolist(),'063')

### Filter Day多日

In [22]:
df_increas.shape
df_increasA.shape
df_increasA.columns[:10]
# df_iA = df_increasA.groupby(level=[0]).tail(1)
df_iA = groupby_count_code(df_increas)
df_iA_multiIndex = df_iA.reset_index().set_index(['code','date'])
df_iA_multiIndex.shape
# df_iA_multiIndex[:10]
# df_iA_multiIndex[-10:]
# df_iA.loc[df_iA.code == '000028']
df_iA_multiIndex[df_iA_multiIndex.index.get_level_values('code') == '000028'][:2]

,,index,name,rate_1,rate_2,rate_3,rate_4,rate_5,rate_6,rate_7,rate_8,...,rate_92,rate_93,rate_94,rate_95,rate_96,rate_97,rate_98,rate_99,rate_100,couts
code,date,,,,,,,,,,,,,,,,,,,,,


In [23]:
df_iAtail = df_iA_multiIndex.groupby(level=[0]).tail(1)
# df7tail.reset_index().code
df_iAtail =  df_iAtail.reset_index().set_index('code')
df_iAtail.loc[:,['date','name','couts']][:5]
# df_iA.loc[:,['date','code','name','couts']][:50]

# df_iAtail[df_iAtail.couts < 3].loc[:,['date','name','couts']]

,date,name,couts
code,,,
600243,2023-09-21,青海华鼎,1
601872,2023-09-21,招商轮船,1
002855,2023-09-21,捷荣技术,1
300504,2023-09-21,天邑股份,1


### show Couts

In [24]:
df_iA.loc[:,['date','code','name','couts']]

,date,code,name,couts
2,2023-09-21,600243,青海华鼎,1
3,2023-09-21,601872,招商轮船,1
0,2023-09-21,002855,捷荣技术,1
1,2023-09-21,300504,天邑股份,1


## cn_stock_indicators MutilIndex

### 读取SQL数据表

In [25]:
# conn = pymysql.connect(**MYSQL_CONN_DBAPI)

# _selcolall = f''' `date`,`code`,`name`,`rate_1` '''
# _selcol = f''' `code`,`name`,`rate_1` '''
_selcol = f''' *'''
_table_name = 'cn_stock_indicators'
# date = last_tddate(1)
today_now = last_tddate(0)
print(today_now)
# conn = pymysql.connect(**MYSQL_CONN_DBAPI)
start_d = last_tddate(5)

sql_now_today = f'''SELECT {_selcol} FROM `{_table_name}` WHERE `date` >= '{start_d}' '''

#mutildate
sql_now_allday = f'''SELECT {_selcol} FROM `{_table_name}` '''

# mycursor2 = pymysql.connect(**MYSQL_CONN_DBAPI)
df2 = panda_df(engine(),sql_now_today)

# df2 = panda_df(engine(),sql_now_allday)



# conn.close()

2023-09-21
(22482, 77)


In [26]:
df2.columns
df2.loc[df2.code =='000061'][:5]

,date,code,name,close,macd,macds,macdh,kdjk,kdjd,kdjj,...,rvi,rvis,fi,force_2,force_13,ene_ue,ene,ene_le,stochrsi_k,stochrsi_d
43,2023-09-14,000061,农 产 品,6.84,0.003391,0.008593,-0.005202,47.7138,43.5225,56.0962,...,0.020792,0.007394,172761.0,17098.1,33136.9,7.58019,6.89729,6.21439,53.6437,56.7608
4539,2023-09-15,000061,农 产 品,6.82,0.001955,0.007406,-0.005451,46.4433,44.4961,50.3377,...,-0.022111,0.017544,-185164.0,-117743.0,1949.2,7.56243,6.88113,6.19983,47.1366,48.4056
9035,2023-09-18,000061,农 产 品,6.76,-0.003951,0.005289,-0.009240,42.3258,43.7727,39.4321,...,-0.052104,-0.002521,-802836.0,-574472.0,-113020.0,7.55133,6.87103,6.19073,28.4712,43.0702
13531,2023-09-19,000061,农 产 品,6.82,-0.003780,0.003565,-0.007345,44.1263,43.8906,44.5978,...,-0.073386,-0.033504,569814.0,188385.0,-15472.6,7.55022,6.87002,6.18982,52.0852,42.5909
18028,2023-09-20,000061,农 产 品,6.87,0.000378,0.002995,-0.002617,49.1145,45.6319,56.0798,...,-0.062559,-0.055941,411980.0,337448.0,45592.0,7.57242,6.89022,6.20802,71.0211,50.5218


In [27]:
df2[ (df2.close >= df2.boll_ub) & (df2.close * 0.95  <= df2.boll_ub) & (df2.macdh > 0)& (df2.macd > 0)& (df2.macds > 0)]
# .loc[:,['rsi_6','rsi_12','rsi_24','rsi']]

,date,code,name,close,macd,macds,macdh,kdjk,kdjd,kdjj,...,rvi,rvis,fi,force_2,force_13,ene_ue,ene,ene_le,stochrsi_k,stochrsi_d
34,2023-09-14,000046,*ST泛海,2.58,0.240521,0.191615,0.048906,92.7322,88.2246,101.7470,...,0.155096,0.173345,18987600.0,18576000.0,8711240.0,2.46864,2.24624,2.02384,97.2637,88.0510
40,2023-09-14,000058,深 赛 格,7.73,0.235000,0.109670,0.125330,68.7638,61.5814,83.1286,...,0.165354,0.102017,-25487600.0,-4692570.0,5119210.0,7.79553,7.09323,6.39093,81.1077,90.8597
91,2023-09-14,000429,粤高速Ａ,8.07,0.101198,0.081554,0.019645,81.3169,74.1953,95.5603,...,0.131633,0.016956,1905760.0,1380440.0,484592.0,8.72460,7.93860,7.15260,100.0000,76.3930
99,2023-09-14,000506,中润资源,5.40,0.153453,0.130519,0.022933,57.9871,50.0217,73.9179,...,0.166092,0.104405,62795000.0,50227100.0,14701500.0,5.55333,5.05303,4.55273,100.0000,52.1102
122,2023-09-14,000536,华映科技,3.65,0.396068,0.236270,0.159798,89.3482,87.7679,92.5089,...,0.364213,0.401461,91033600.0,90489600.0,50798300.0,3.17127,2.88557,2.59987,86.8899,84.8038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22122,2023-09-20,603557,ST起步,2.60,0.015187,0.000855,0.014332,74.9307,69.2501,86.2919,...,0.077810,0.023075,792840.0,546964.0,145854.0,2.76723,2.51793,2.26863,100.0000,100.0000
22124,2023-09-20,603559,ST通脉,7.55,0.111081,0.032707,0.078374,69.9732,57.7877,94.3441,...,0.270270,0.131034,728196.0,1385480.0,788801.0,7.75446,7.05586,6.35726,100.0000,100.0000
22285,2023-09-20,603863,松炀资源,22.60,0.348027,0.094791,0.253236,85.1396,78.4694,98.4800,...,0.238780,0.126549,7747190.0,5522120.0,2283040.0,23.08580,21.00600,18.92620,100.0000,92.2164
22330,2023-09-20,603933,睿能科技,20.36,0.642599,0.290827,0.351772,77.1538,71.5254,88.4108,...,0.380751,0.365444,40904900.0,26445500.0,9368200.0,20.46620,18.62240,16.77860,89.3296,75.6186


# inStock 序列化数据测试

## 定义参数

In [28]:
stock_hist_cache_path='Y:\\dockerf\\stock\\instockcache\\hist\\2020'+get_today(sep='')[-4:-2]+'\\2020'+get_today(sep='')[-4:]
stock_hist_cache_path

'Y:\\dockerf\\stock\\instockcache\\hist\\202009\\20200921'

In [29]:
import logging
import os.path
#  import os.sep
import datetime
import numpy as np
import pandas as pd
import talib as tl

os.environ['NUMEXPR_MAX_THREADS'] = '12'
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
adjust='qfq'
get_today()
# stock_hist_cache_path='Y:\\dockerf\\stock\\instockcache\\hist\\202005\\2020'+get_today(sep='')[-4:]
code = '601336'
cache_file = os.path.join(stock_hist_cache_path, "%s%s.gzip.pickle" % (code, adjust))
print(cache_file)
df = pd.read_pickle(cache_file, compression="gzip")


Y:\dockerf\stock\instockcache\hist\202009\20200921\601336qfq.gzip.pickle


FileNotFoundError: [Errno 2] No such file or directory: 'Y:\\dockerf\\stock\\instockcache\\hist\\202009\\20200921\\601336qfq.gzip.pickle'

In [ ]:
df.shape
df[-2:]
# df.columns
lastday=df.date.values[-1]
lastday
data = df.copy()
# macd
data.loc[:, 'diff'], data.loc[:, 'dea'], data.loc[:, 'macd'] = tl.MACD(
    data['close'].values, fastperiod=12, slowperiod=26, signalperiod=9)
data['diff'].values[np.isnan(data['diff'].values)] = 0.0
data['dea'].values[np.isnan(data['dea'].values)] = 0.0
data['macd'].values[np.isnan(data['macd'].values)] = 0.0

data.loc[:, 'boll_ub'], data.loc[:, 'boll'], data.loc[:, 'boll_lb'] = tl.BBANDS \
    (data['close'].values, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
data['boll_ub'].values[np.isnan(data['boll_ub'].values)] = 0.0
data['boll'].values[np.isnan(data['boll'].values)] = 0.0
data['boll_lb'].values[np.isnan(data['boll_lb'].values)] = 0.0
data.iloc[-60:]['macd'].max()
# DIFF:=EMA(CLOSE,12) - EMA(CLOSE,26);  256 
# DEA := EMA(DIFF,9); 1.52
# MACD := 2*(DIFF-DEA); 2.08
print(not False)

## stock_period_to_typ Resample 'W-FRI'

In [ ]:
def get_tdx_stock_period_to_type(stock_data, period_day='W-FRI', periods=5, ncol=None, ratiodays=True):
    """_周期转换周K,月K_

    Returns:
        _type_: _description_
    """
    #快速日期处理
    #https://www.likecs.com/show-204682607.html

    period_type = period_day
    if 'date' in stock_data.columns:
        stock_data.set_index('date', inplace=True)
    stock_data['date'] = stock_data.index
    lastday = stock_data.date.values[-1]
    lastday2 = stock_data.date.values[-2]
    indextype = True if stock_data.index.dtype == 'datetime64[ns]' else False
    # if cct.get_work_day_status() and 915 < cct.get_now_time_int() < 1500:
    #     stock_data = stock_data[stock_data.index < cct.get_today()]

    if stock_data.index.name == 'date':
        stock_data.index = pd.to_datetime(stock_data.index, format='%Y-%m-%d')
    elif 'date' in stock_data.columns:
        stock_data.set_index('date', inplace=True)
        stock_data.sort_index(ascending=True, inplace=True)
        stock_data.index = pd.to_datetime(stock_data.index, format='%Y-%m-%d')
    # else:
    #     log.error("index.name not date,pls check:%s" % (stock_data[:1]))

    period_stock_data = stock_data.resample(period_type).last()
    # period_stock_data['percent']=stock_data['percent'].resample(period_type,how=lambda x:(x+1.0).prod()-1.0)
    # print stock_data.index[0],stock_data.index[-1]
    # period_stock_data.index =
    # pd.DatetimeIndex(start=stock_data.index.values[0],end=stock_data.index.values[-1],freq='BM')

    period_stock_data['open'] = stock_data[
        'open'].resample(period_type).last()
    period_stock_data['high'] = stock_data[
        'high'].resample(period_type).max()
    period_stock_data['low'] = stock_data[
        'low'].resample(period_type).min()
    
    lastWeek1 = str(period_stock_data['open'].index.values[-1])[:10]
    lastweek2 = str(period_stock_data['open'].index.values[-2])[:10]
    if ratiodays:
        if period_day == 'W-FRI':
            print(lastWeek1,lastweek2,lastday,lastday2)
            duratio = int(str(datetime.datetime.strptime(lastWeek1, '%Y-%m-%d').date() - datetime.datetime.strptime(lastday, '%Y-%m-%d').date())[0])
            ratio_d =(5-(duratio%5))/5
            print("ratio_d:%s %s"%(ratio_d,lastday))
        elif period_day.find('W') >= 0:
            print(lastWeek1,lastweek2,lastday,lastday2)
            duratio = int(str(datetime.datetime.strptime(lastday, '%Y-%m-%d').date() - datetime.datetime.strptime(lastweek2, '%Y-%m-%d').date())[0])
            ratio_d =(duratio)/5
            print("ratio_d:%s %s"%(ratio_d,lastday))
        elif period_day == 'BM':
            # daynow = '2023-04-26'
            # lastday = '2023-04-23'
            print(lastWeek1,lastweek2,lastday,lastday2)
            # print((str(datetime.datetime.strptime(lastWeek1, '%Y-%m-%d').date() - datetime.datetime.strptime(lastday, '%Y-%m-%d').date())[:2]))
            duratio = int(str(datetime.datetime.strptime(lastWeek1, '%Y-%m-%d').date() - datetime.datetime.strptime(lastday, '%Y-%m-%d').date())[:2])
            ratio_d =(30-(duratio%30))/30
            print("ratio_d:%s %s dura:%s"%(ratio_d,lastday,duratio))
        elif period_day.find('M') >= 0:
            ratio_d = 1
            
    else:
        ratio_d = 1
        print(ratio_d)
        
    if ncol is not None:
        for co in ncol:
            period_stock_data[co] = stock_data[co].resample(period_type).sum()
            if ratiodays:
                period_stock_data[co] = period_stock_data[co].apply(lambda x: round(x / ratio_d, 1))
                
    # else:
    period_stock_data['amount'] = stock_data[
        'amount'].resample(period_type).sum()
    period_stock_data['volume'] = stock_data[
        'volume'].resample(period_type).sum()
    if ratiodays and ratio_d != 1:
        period_stock_data['amount'] = period_stock_data['amount'].apply(lambda x: round(x / ratio_d, 1))
        period_stock_data['volume'] = period_stock_data['volume'].apply(lambda x: round(x / ratio_d, 1))
                
    # period_stock_data['turnover']=period_stock_data['vol']/(period_stock_data['traded_market_value'])/period_stock_data['close']
    period_stock_data.index = stock_data['date'].resample(period_type).last().index
    # print period_stock_data.index[:1]
    if 'code' in period_stock_data.columns:
        period_stock_data = period_stock_data[period_stock_data['code'].notnull()]
    period_stock_data = period_stock_data.dropna()
    # period_stock_data.reset_index(inplace=True)
    # period_stock_data.set_index('date',inplace=True)
    # print period_stock_data.columns,period_stock_data.index.name
    # and period_stock_data.index.dtype != 'datetime64[ns]')
    if not indextype and period_stock_data.index.name == 'date':
        # stock_data.index = pd.to_datetime(stock_data.index, format='%Y-%m-%d')
        period_stock_data.index = [str(x)[:10] for x in period_stock_data.index]
        period_stock_data.index.name = 'date'
    else:
        if 'date' in period_stock_data.columns:
            period_stock_data = period_stock_data.drop(['date'], axis=1)
    # print period_stock_data
    return period_stock_data


### Resample Week W-FRI

In [ ]:
# dfw = get_tdx_stock_period_to_type(data.copy(), period_day='W-FRI', ncol=['turnover', 'amplitude', 'quote_change'],ratiodays=False)
# dfw[-3:]
dfw = get_tdx_stock_period_to_type(data.copy(), period_day='W-FRI', ncol=['turnover', 'amplitude', 'quote_change'],ratiodays=True)
dfw[-3:]
dfw = get_tdx_stock_period_to_type(data.copy(), period_day='W', ncol=['turnover', 'amplitude', 'quote_change'],ratiodays=True)
dfw[-3:]
# data[-5:]

### Resample Month BM
https://note.nkmk.me/python-pandas-time-series-freq/

    BM: 月末最終営業日ごと
    MS: 毎月の開始日ごと
    BMS: 毎月の最初の営業日ごと


In [ ]:
dfw = get_tdx_stock_period_to_type(data.copy(), period_day='BM', ncol=['turnover', 'amplitude', 'quote_change'],ratiodays=False)
dfw[-2:]
dfw = get_tdx_stock_period_to_type(data.copy(), period_day='BM', ncol=['turnover', 'amplitude', 'quote_change'],ratiodays=True)
dfw[-3:]

In [ ]:
daynow=dfw.index.values[-1]
str(datetime.datetime.strptime(daynow, '%Y-%m-%d').date() - datetime.datetime.strptime(lastday, '%Y-%m-%d').date())[0]
int(str(datetime.datetime.strptime(daynow, '%Y-%m-%d').date() - datetime.datetime.strptime(lastday, '%Y-%m-%d').date())[0])


# inStock RSI

In [ ]:
import logging
import os.path
#  import os.sep
import datetime
import numpy as np
import pandas as pd
import talib as tl

os.environ['NUMEXPR_MAX_THREADS'] = '12'
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
adjust='qfq'
# stock_hist_cache_path='Y:\\dockerf\\stock\\instockcache\\hist\\202005\\20200530'
code = '000002'
cache_file = os.path.join(stock_hist_cache_path, "%s%s.gzip.pickle" % (code, adjust))
print(cache_file)
df = pd.read_pickle(cache_file, compression="gzip")

In [ ]:
df.shape
df[-5:]
df[:1]
# df.columns
lastday=df.date.values[-1]
lastday

In [ ]:
data=df.copy()

In [ ]:
def check_rsi_status(df,threshold=60,period_day=False):
    # macd
    data = df.copy()
    # cci 计算方法和结果和stockstats不同，stockstats典型价采用均价(总额/成交量)计算
    data.loc[:, 'cci'] = tl.CCI(data['high'].values, data['low'].values, data['close'].values, timeperiod=14)
    data['cci'].values[np.isnan(data['cci'].values)] = 0.0
    # data.loc[:, 'cci_84'] = tl.CCI(data['high'].values, data['low'].values, data['close'].values, timeperiod=84)
    # data['cci_84'].values[np.isnan(data['cci_84'].values)] = 0.0
    rsilast5 = data.iloc[-6:-1]['cci']
    rsilast5max = rsilast5.max()
    rsilast = data.iloc[-1]['cci']

    if rsilast5max < 0 and rsilast > rsilast5max and rsilast5.iloc[-1] < rsilast5max:
        return True
    else:
        return False

In [ ]:
# data.loc[:, 'cci'] = tl.CCI(data['high'].values, data['low'].values, data['close'].values, timeperiod=14)
# data['cci'].values[np.isnan(data['cci'].values)] = 0.0
# # data.loc[:, 'cci_84'] = tl.CCI(data['high'].values, data['low'].values, data['close'].values, timeperiod=84)
# # data['cci_84'].values[np.isnan(data['cci_84'].values)] = 0.0
# rsilast5 = data.iloc[-6:-1]['cci']
# rsilast5max = rsilast5.max()
# rsilast = data.iloc[-1]['cci']
# data['cci'][-6:]
# rsilast5.iloc[-1]
# rsilast5,rsilast
# rsilast5max < 0 and rsilast > rsilast5max and rsilast5.iloc[-1] < rsilast5max
check_rsi_status(df)